In [2]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model("modelAdditionalLayers5block.keras")  # Change this to your model path

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
labels = [chr(i) for i in range(ord('A'), ord('Z')+1)]  # ['A', 'B', ..., 'Z']

def draw_hand_landmarks_on_black(hand_landmarks, image_size=128):
    # Extract landmark coordinates
    landmarks = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark])

    # Normalize coordinates
    x_vals = landmarks[:, 0]
    y_vals = landmarks[:, 1]
    min_x, max_x = np.min(x_vals), np.max(x_vals)
    min_y, max_y = np.min(y_vals), np.max(y_vals)

    width = max_x - min_x
    height = max_y - min_y
    scale = 0.8 * image_size / max(width, height)

    # Apply scale and center
    landmarks[:, 0] = (landmarks[:, 0] - min_x) * scale
    landmarks[:, 1] = (landmarks[:, 1] - min_y) * scale

    offset_x = (image_size - width * scale) / 2
    offset_y = (image_size - height * scale) / 2
    landmarks[:, 0] += offset_x
    landmarks[:, 1] += offset_y

    # Create black canvas (now with 3 channels for color)
    black_img = np.zeros((image_size, image_size, 3), dtype=np.uint8)
    landmark_points = landmarks.astype(np.int32)

    # Draw connections (green, thinner lines)
    for connection in mp_hands.HAND_CONNECTIONS:
        start = tuple(landmark_points[connection[0]])
        end = tuple(landmark_points[connection[1]])
        cv2.line(black_img, start, end, (0, 255, 0), 1)  # Green, thickness=1

    # Draw landmark points (green, smaller points)
    for point in landmark_points:
        cv2.circle(black_img, tuple(point), 1, (0, 255, 0), -1)  # Green, radius=1

    return black_img

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    prediction_text = ""

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Preprocess hand to match training input
            colored_img = draw_hand_landmarks_on_black(hand_landmarks, image_size=128)
            
            # Convert to grayscale for model input
            gray_img = cv2.cvtColor(colored_img, cv2.COLOR_BGR2GRAY)
            normalized_img = gray_img.astype("float32") / 255.0
            input_img = np.expand_dims(normalized_img, axis=(0, -1))  # Shape: (1, 128, 128, 1)

            # Predict
            prediction = model.predict(input_img, verbose=0)
            predicted_index = np.argmax(prediction)
            confidence = prediction[0][predicted_index]

            if confidence > 0.8:
                prediction_text = f"Prediction: {labels[predicted_index]} ({confidence*100:.1f}%)"
            else:
                prediction_text = "Unsure"

            # Optional: draw hand landmarks on webcam feed
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Show model input for debug (now shows green landmarks)
            cv2.imshow("Model Input", colored_img)

            break  # Only use the first hand

    # Display the prediction
    cv2.putText(frame, prediction_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("ASL Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
#MobileNetV2Architecture Live Test
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# Load your trained model (MobileNetV2 expects RGB input with 3 channels)
model = tf.keras.models.load_model("MobileNetV21.keras")

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

# List of labels (A-Z)
labels = [chr(i) for i in range(ord('A'), ord('Z')+1)]  # ['A', 'B', ..., 'Z']

def draw_hand_landmarks_on_black(hand_landmarks, image_size=128):
    # Extract landmark coordinates
    landmarks = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark])

    # Normalize coordinates to fit within image size
    x_vals = landmarks[:, 0]
    y_vals = landmarks[:, 1]
    min_x, max_x = np.min(x_vals), np.max(x_vals)
    min_y, max_y = np.min(y_vals), np.max(y_vals)

    width = max_x - min_x
    height = max_y - min_y
    scale = 0.8 * image_size / max(width, height)

    landmarks[:, 0] = (landmarks[:, 0] - min_x) * scale
    landmarks[:, 1] = (landmarks[:, 1] - min_y) * scale

    offset_x = (image_size - width * scale) / 2
    offset_y = (image_size - height * scale) / 2
    landmarks[:, 0] += offset_x
    landmarks[:, 1] += offset_y

    # Create black canvas (RGB)
    black_img = np.zeros((image_size, image_size, 3), dtype=np.uint8)
    landmark_points = landmarks.astype(np.int32)

    # Draw connections and points
    for connection in mp_hands.HAND_CONNECTIONS:
        start = tuple(landmark_points[connection[0]])
        end = tuple(landmark_points[connection[1]])
        cv2.line(black_img, start, end, (0, 255, 0), 1)

    for point in landmark_points:
        cv2.circle(black_img, tuple(point), 1, (0, 255, 0), -1)

    return black_img

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    prediction_text = ""

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get black canvas with green hand drawing
            input_image = draw_hand_landmarks_on_black(hand_landmarks, image_size=128)

            # Normalize and expand dims for MobileNetV2
            normalized_img = input_image.astype("float32") / 255.0
            input_tensor = np.expand_dims(normalized_img, axis=0)  # Shape: (1, 128, 128, 3)

            # Predict
            prediction = model.predict(input_tensor, verbose=0)
            predicted_index = np.argmax(prediction)
            confidence = prediction[0][predicted_index]

            if confidence > 0.8:
                prediction_text = f"Prediction: {labels[predicted_index]} ({confidence*100:.1f}%)"
            else:
                prediction_text = "Unsure"

            # Draw landmarks on original frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Show model input for debugging
            cv2.imshow("Model Input", input_image)

            break  # Only process one hand

    # Show prediction on webcam feed
    cv2.putText(frame, prediction_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("ASL Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model("model11.keras")
labels = [chr(i) for i in range(ord('A'), ord('Z')+1)]

# ---------- Replace with path to your image ----------
image_path = 'adcv.png'
colored_img = cv2.imread(image_path)

if colored_img is None:
    print("Error: Could not load image.")
else:
    # Convert to grayscale and preprocess (same as training)
    gray_img = cv2.cvtColor(colored_img, cv2.COLOR_BGR2GRAY)
    normalized_img = gray_img.astype("float32") / 255.0
    input_img = np.expand_dims(normalized_img, axis=(0, -1))  # Add batch and channel dims

    # Predict
    prediction = model.predict(input_img, verbose=0)
    predicted_index = np.argmax(prediction)
    confidence = prediction[0][predicted_index]

    if confidence > 0.5:
        prediction_text = f"Prediction: {labels[predicted_index]} ({confidence*100:.1f}%)"
    else:
        prediction_text = "Unsure"

    # Show results
    print(prediction_text)
    cv2.imshow("Model Input", colored_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

ValueError: File not found: filepath=model11.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
print('jsms')